In [2]:
import sqlite3
import sys
import requests_cache
import pandas as pd
import requests
import random
import string
from bs4 import NavigableString, Comment
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver import ChromeOptions

In [3]:
sys.path.insert(0, "/Users/samdillard/python/github.com/samhld/nba_stats/app/")
requests_cache.install_cache()

In [4]:
from scraper import getters
from preprocessor import builders
from PandasBasketball.stats import player_stats
from PandasBasketball.stats import get_data_master
from PandasBasketball.errors import TableNonExistent
from preprocessor.builders import create_player_stats_df

In [4]:
# conn = sqlite3.connect("../app/dbs/all_players.db")
# c = conn.cursor()

In [5]:
# master = """SELECT name FROM sqlite_master WHERE type='table'"""
# c.execute(master).fetchall()

In [6]:
all_players = '''SELECT PTS FROM players_per_game WHERE Player = "LeBron James"'''
# c.execute(all_players).fetchall()

In [7]:
rename_table = '''ALTER TABLE `players_per_game` RENAME TO `players_per36`'''
# c.execute(rename_table)

In [8]:
# master = """SELECT name FROM sqlite_master WHERE type='table'"""
# c.execute(master).fetchall()

In [9]:
def _get_player_tables(url, stat):
    
    one_header_tables = ["totals", "per_minute", "per_poss", "advanced",
                        "playoffs_per_game", "playoffs_totals", "playoffs_per_minute",
                        "playoffs_per_poss", "playoffs_advanced"]
    
    two_header_tables = ["adj_shooting", "playoffs_shooting", "shooting", "pbp", "playoffs_pbp"]
    
    res = requests.get(url)
    
    if stat == "per_game":
        
        soup = bs(res.text, "lxml")
        table = soup.find("table", id="per_game")
        
    elif stat in one_header_tables or stat in two_header_tables:

        soup = bs(res.text, "lxml")
        comment_table = soup.find(text=lambda x: isinstance(x, NavigableString) and f'id="{stat}"' in x)
        soup = bs(comment_table, "lxml")
        table = soup.find("table", id=stat)
        
        if stat in two_header_tables:
            
            table.find("tr").decompose()

    else:
        raise TableNonExistent
    return table
        

In [6]:
from scraper.getters import _get_player_table
bron_url = "https://www.basketball-reference.com/players/j/jamesle01.html"
_get_player_table(bron_url, "totals")

<table class="row_summable sortable stats_table" data-cols-to-freeze="1,3" id="totals">
<caption>Totals Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr>
<th aria-label="If listed as single number, the year the season ended.★ - Indicates All-Star for league.Only on regular season tables." class="poptip sort_default_asc center" data-stat="season" data-tip="If listed as single number, the year the season ended.&lt;br&gt;★ - Indicates All-Star for league.&lt;br&gt;Only on regular season tables." scope="col">Season</th>
<th aria-label="Player's age on February 1 of the season" class="poptip sort_default_asc center" data-stat="age" data-tip="Player's age on February 1 of the season" scope="col">Age</th>
<th aria-label="Team" class="poptip sort_default_asc center" data-stat="team_id" data-tip="Team" sc

In [382]:
# no selenium way

res = requests.get(bron_url)
soup = bs(res.text, "html.parser")
comment_table = soup.find(text=lambda x: isinstance(x, NavigableString) and "id=shooting" in x)
comment_table
# soup = BeautifulSoup(comment_table, "lxml")
# table = soup.find("table")
# table

In [383]:
# chromedriver = "/Users/samdillard/Downloads/chromedriver"
# options = ChromeOptions()
# options.add_argument('--headless')
# driver = webdriver.Chrome(chromedriver, options=options)
# soup = bs(driver.page_source,'lxml')
# driver.quit()
# tables = soup.find_all('table')
# tables

In [10]:
bron_url = "https://www.basketball-reference.com/players/j/jamesle01.html"

In [402]:
def get_all_player_tables(url):
    
    supported_tables = ["totals", "per_minute", "per_poss", "advanced",
                        "playoffs_per_game", "playoffs_totals", "playoffs_per_minute",
                        "playoffs_per_poss", "playoffs_advanced", "adj_shooting", 
                        "playoffs_shooting", "shooting", "pbp", "playoffs_pbp"]
    
    tables = []
    for table in supported_tables:
        print(table)
        tables.append(_get_player_tables(url, table))
        
    return tables
        

In [403]:
bron_tables = get_all_player_tables(bron_url)
len(bron_tables)

totals
per_minute
per_poss
advanced
playoffs_per_game
playoffs_totals
playoffs_per_minute
playoffs_per_poss
playoffs_advanced
adj_shooting
playoffs_shooting
shooting
pbp
playoffs_pbp


14

### Get Trp Dbl column from Totals table

In [266]:
# body = table.find("tbody")
# rows = body.find_all("tr")
# header = table.find("thead").find("tr")
# header_cols = []
# for head in header.find_all("th"):
#     header_cols.append(head.text)
# # header_cols

# non_header_cols = [td['data-stat'] for td in rows[0].find_all("td")]
# header_cols
# # print(len(header_cols), len(non_header_cols))
df_totals = get_data_master(totals_table, "player")
df_totals

,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,,Trp Dbl
0,2003-04,19,CLE,NBA,SG,79,79,3122,622,1492,...,333,432,465,130,58,273,149,1654,,0
1,2004-05,20,CLE,NBA,SF,80,80,3388,795,1684,...,477,588,577,177,52,262,146,2175,,4
2,2005-06,21,CLE,NBA,SF,79,79,3361,875,1823,...,481,556,521,123,66,260,181,2478,,5
3,2006-07,22,CLE,NBA,SF,78,78,3190,772,1621,...,443,526,470,125,55,250,171,2132,,1
4,2007-08,23,CLE,NBA,SF,75,74,3027,794,1642,...,459,592,539,138,81,255,165,2250,,7
5,2008-09,24,CLE,NBA,SF,81,81,3054,789,1613,...,507,613,587,137,93,241,139,2304,,7
6,2009-10,25,CLE,NBA,SF,76,76,2966,768,1528,...,483,554,651,125,77,261,119,2258,,4
7,2010-11,26,MIA,NBA,SF,79,79,3063,758,1485,...,510,590,554,124,50,284,163,2111,,4
8,2011-12,27,MIA,NBA,SF,62,62,2326,621,1169,...,398,492,387,115,50,213,96,1683,,0
9,2012-13,28,MIA,NBA,PF,76,76,2877,765,1354,...,513,610,551,129,67,226,110,2036,,4


In [ ]:
url = "https://www.basketball-reference.com/players/j/jamesle01.html"
totals_table = get_player_tables(url, "totals")

In [289]:
cols = [td['data-stat'] for td in rows[0].find_all("td")]
season = ["season"]
season.extend(cols)
season

['season',
 'age',
 'team_id',
 'lg_id',
 'pos',
 'g',
 'gs',
 'mp',
 'fg',
 'fga',
 'fg_pct',
 'fg3',
 'fg3a',
 'fg3_pct',
 'fg2',
 'fg2a',
 'fg2_pct',
 'efg_pct',
 'ft',
 'fta',
 'ft_pct',
 'orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 'DUMMY',
 'trp_dbl']

In [200]:
nums = [1]
nums2 = list(range(10))
nums.extend(nums2)